In [1]:
import sframe
products = sframe.SFrame('amazon_baby.gl/')

[INFO] sframe.cython.cy_server: SFrame v2.1 started. Logging /tmp/sframe_server_1480270519.log


In [2]:
def remove_punctuation(text):
    import string
    return text.translate(None, string.punctuation) 

In [3]:
products['review_clean'] = products['review'].apply(remove_punctuation)
products = products.fillna('review', '')  # fill in N/A's in the review column
products = products[products['rating'] != 3]
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
train_data, test_data = products.random_split(.8, seed=1)

In [6]:
len(train_data)

133416

In [8]:
train_data

name,review,rating,review_clean,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,it came early and was notdisappointed i love ...,1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,Very soft and comfortableand warmer than it ...,1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,This is a product wellworth the purchase I ...,1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,All of my kids have criednonstop when I tried to ...,1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,When the Binky Fairy cameto our house we didnt ...,1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,Lovely book its boundtightly so you may no ...,1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,Perfect for new parentsWe were able to keep ...,1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,A friend of mine pinnedthis product on Pinte ...,1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This book is perfect!I'm a first time new ...,5.0,This book is perfect Ima first time new mom and ...,1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I originally just gavethe nanny a pad of paper ...,4.0,I originally just gavethe nanny a pad of paper ...,1


In [7]:
len(test_data)

33336

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
     # Use this token pattern to keep single-letter words
# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform(train_data['review_clean'])
# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(test_data['review_clean'])

In [9]:
train_matrix

<133416x121712 sparse matrix of type '<type 'numpy.int64'>'
	with 7326618 stored elements in Compressed Sparse Row format>

In [5]:
from sklearn import linear_model
sentiment_model = linear_model.LogisticRegression(n_jobs=-1)
sentiment_model.fit(train_matrix, train_data['sentiment'])
print("Number of coefficients in the sentiment model %f" % len(sentiment_model.coef_[0]))

Number of coefficients in the sentiment model 121712.000000
